# Pré-processamento de Dados - Senti-Pred

Este notebook contém as etapas de pré-processamento dos dados para o projeto Senti-Pred.

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Download dos recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Carregar os dados
data_path = '../data/raw/test.csv'
df = pd.read_csv(data_path)

# Exibir as primeiras linhas
df.head()

In [ ]:
# Funções de pré-processamento
def clean_text(text):
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        # Remover URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Remover menções e hashtags
        text = re.sub(r'@\w+|#\w+', '', text)
        # Remover pontuação e caracteres especiais
        text = re.sub(r'[^\w\s]', '', text)
        # Remover números
        text = re.sub(r'\d+', '', text)
        # Remover espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ''

def remove_stopwords(text):
    if isinstance(text, str):
        stop_words = set(stopwords.words('portuguese'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]
        return ' '.join(filtered_text)
    return ''

def lemmatize_text(text):
    if isinstance(text, str):
        lemmatizer = WordNetLemmatizer()
        word_tokens = word_tokenize(text)
        lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
        return ' '.join(lemmatized_text)
    return ''

In [ ]:
# Aplicar pré-processamento
if 'text' in df.columns:
    # Criar cópia do dataframe original
    df_processed = df.copy()
    
    # Aplicar limpeza de texto
    df_processed['text_clean'] = df_processed['text'].apply(clean_text)
    
    # Remover stopwords
    df_processed['text_no_stopwords'] = df_processed['text_clean'].apply(remove_stopwords)
    
    # Lematização
    df_processed['text_lemmatized'] = df_processed['text_no_stopwords'].apply(lemmatize_text)
    
    # Exibir exemplos de textos processados
    sample_df = pd.DataFrame({
        'Original': df_processed['text'].iloc[:5],
        'Limpo': df_processed['text_clean'].iloc[:5],
        'Sem Stopwords': df_processed['text_no_stopwords'].iloc[:5],
        'Lematizado': df_processed['text_lemmatized'].iloc[:5]
    })
    
    sample_df

In [ ]:
# Salvar dados processados
processed_path = '../data/processed/processed_data.csv'
os.makedirs(os.path.dirname(processed_path), exist_ok=True)
df_processed.to_csv(processed_path, index=False)
print(f"Dados processados salvos em: {processed_path}")

## Análise dos Dados Processados

Vamos analisar algumas estatísticas dos dados após o pré-processamento.

In [ ]:
if 'text' in df.columns and 'text_lemmatized' in df_processed.columns:
    # Comprimento dos textos antes e depois do processamento
    df_processed['original_length'] = df_processed['text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
    df_processed['processed_length'] = df_processed['text_lemmatized'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
    
    # Visualizar a redução no comprimento dos textos
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.histplot(df_processed['original_length'], bins=30, kde=True, color='blue')
    plt.title('Distribuição do Comprimento Original')
    plt.xlabel('Número de Palavras')
    plt.ylabel('Frequência')
    
    plt.subplot(1, 2, 2)
    sns.histplot(df_processed['processed_length'], bins=30, kde=True, color='green')
    plt.title('Distribuição do Comprimento Após Processamento')
    plt.xlabel('Número de Palavras')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas da redução
    reduction = ((df_processed['original_length'] - df_processed['processed_length']) / df_processed['original_length']) * 100
    print(f"Redução média no comprimento do texto: {reduction.mean():.2f}%")

## Conclusões do Pré-processamento

- Resumo das transformações aplicadas
- Impacto do pré-processamento nos dados
- Próximos passos para a modelagem